In [ ]:
from openai import OpenAI
import PyPDF2
import os
import base64
from typing import Optional
import fitz  # PyMuPDF
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pypdf2 pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 52.8 MB/s eta 0:00:00


In [ ]:
# https://github.com/meta-llama/llama-cookbook/blob/main/end-to-end-use-cases/whatsapp_llama_4_bot/ec2_services.py
def get_llm_response(text_input: str, image_input : str = None) -> str:
    """
    Get the response from the Together AI LLM given a text input and an optional image input.

    Args:
        text_input (str): The text to be sent to the LLM.
        image_input (str, optional): The base64 encoded image to be sent to the LLM. Defaults to None.

    Returns:
        str: The response from the LLM.
    """
    messages = []
    # print(bool(image_input))
    if image_input:
        messages.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_input}"}
        })
    messages.append({
        "type": "text",
        "text": text_input
    })
    try:
        #client = Together(api_key=TOGETHER_API_KEY)
        client = OpenAI(
            api_key="LLM|1092127122939929|swnut7Dzo4N-CdXCmXFLKxWJC9s",
            base_url= "https://api.llama.com/compat/v1/")
        completion = client.chat.completions.create(
            model="Llama-4-Maverick-17B-128E-Instruct-FP8",
            messages=[
                {
                    "role": "user",
                    "content": messages
                }
            ]
        )

        if completion.choices and len(completion.choices) > 0:
            return completion.choices[0].message.content
        else:
            print("Empty response from Together API")
            return None
    except Exception as e:
        print(f"LLM error: {e}")
        return None

# https://github.com/meta-llama/llama-cookbook/blob/main/end-to-end-use-cases/NotebookLlama/Step-1%20PDF-Pre-Processing-Logic.ipynb
def validate_pdf(file_path: str) -> bool:
    if not os.path.exists(file_path):
        print(f"Error: File not found at path: {file_path}")
        return False
    if not file_path.lower().endswith('.pdf'):
        print("Error: File is not a PDF")
        return False
    return True


def extract_text_from_pdf(file_path: str, max_chars: int = -1) -> Optional[str]:
    # max_chars = -1 for no text length limit
    if not validate_pdf(file_path):
        return None

    try:
        with open(file_path, 'rb') as file:
            # Create PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)

            # Get total number of pages
            num_pages = len(pdf_reader.pages)
            print(f"Processing PDF with {num_pages} pages...")

            extracted_text = []
            total_chars = 0

            # Iterate through all pages
            for page_num in tqdm(range(num_pages)):
                # Extract text from page
                page = pdf_reader.pages[page_num]
                text = page.extract_text()

                # Check if adding this page's text would exceed the limit
                if max_chars != -1 and total_chars + len(text) > max_chars:
                    # Only add text up to the limit
                    remaining_chars = max_chars - total_chars
                    extracted_text.append(text[:remaining_chars])
                    print(f"Reached {max_chars} character limit at page {page_num + 1}")
                    break

                extracted_text.append(text)
                total_chars += len(text)
                # print(f"Processed page {page_num + 1}/{num_pages}")

            final_text = '\n'.join(extracted_text)
            print(f"\nExtraction complete! Total characters: {len(final_text)}")
            return final_text

    except PyPDF2.PdfReadError:
        print("Error: Invalid or corrupted PDF file")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return None


def encode_image_to_base64(image_path):
    """
    Encode an image to Base64.

    :param image_path: Path to the image file.
    :return: Base64 encoded string of the image.
    """
    try:
        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read())
            # To return the encoded string as a string (not bytes), decode it to UTF-8
            return encoded_string.decode('utf-8')
    except FileNotFoundError:
        print(f"The file {image_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def extract_images_from_pdf(pdf_path, output_folder):
    """
    Extract images from a PDF.

    :param pdf_path: Path to the PDF file.
    """
    try:
        output_fps = []
        # Open the PDF
        doc = fitz.open(pdf_path)

        # Iterate through the pages
        for page_index in range(len(doc)):
            page = doc[page_index]
            image_list = page.get_images()

            # Iterate through the images on the page
            for img_index, img in enumerate(image_list):
                xref = img[0]
                # Extract the image
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]

                # Save the image
                image_path = os.path.join(
                    output_folder, f"image_page_{page_index+1}_{img_index+1}.png")
                with open(image_path, "wb") as image_file:
                    image_file.write(image_bytes)

                # print(f"Image saved to {image_path}")
                output_fps.append(image_path)

        doc.close()
        return output_fps
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/bingham_images'
image_path = os.path.join(folder_path, '1.png')  # Update this to the path of your image
encoded_image = encode_image_to_base64(image_path)
get_llm_response("explan this image", encoded_image)

'The image presents a geological map of the Bingham porphyry deposit, showcasing the location of faults, intrusions, and copper and gold grades. The map is based on unpublished mine geology maps by A.J. Maughan and A.J. Swenson (1990), and Utah Copper geology staff in prior years, 1:600 scale open-pit mapping by Redmond (2002) within the box labeled "QMP-LP zone, Fig. 3"), and Atkinson and Einaudi (1978).\n\n**Map Details:**\n\n*   **Scale:** 500m\n*   **Orientation:** North is indicated by an arrow at the top center of the map.\n*   **Legend:**\n    *   Quartz latite porphyry (QLP)\n    *   Quartz latite porphyry breccia (QLPbx)\n    *   Biotite porphyry (BP)\n    *   Latite porphyry (LP)\n    *   Quartz monzonite porphyry (QMP)\n    *   Equigranular monzonite\n    *   Quartzite\n    *   Siltstone & Limestone\n    *   Skarn & Limestone\n    *   Fault with dip symbol\n    *   Fold axis, with plunge, anticline, syncline\n    *   Strike and dip of beds, inclined, overturned\n\n**Key Feat

In [ ]:
pdf_fp = os.path.join(folder_path, 'The_Bingham_Canyon_Porphyry_Cu_Mo_Au_Dep.pdf')
pdf_text = extract_text_from_pdf(pdf_fp)
pdf_text

Processing PDF with 26 pages...


100%|██████████| 26/26 [00:02<00:00, 10.63it/s]



Extraction complete! Total characters: 103402


"0361-0128/10/3864/43-26 43Introduction\nTHE GENETIC link between porphyry copper deposits and\nmagmatic-hydrothermal processes is well established, based\non mapping and petrological studies (Gustafson and Hunt,\n1975; Dilles and Einaudi, 1992; Proffett, 2003), characteris-\ntics of fluid inclusions (Roedder, 1971; Nash, 1976; Eastoe,\n1978; Bodnar, 1995; Heinrich el al., 1999; Ulrich et al., 1999,\n2002; Audetat et al., 2008), light stable isotopes (Sheppard et\nal., 1969, 1971; Harris et al., 2005), and analogy with active\nvolcanic fumaroles and geothermal systems (Henley and Mc-\nNabb, 1978; Hedenquist and Lowenstern, 1994; Einaudi et\nal., 2003). Formation of porphyry copper deposits requires\nthe escape of magma and hydrothermal fluid from a magma\nchamber located well below the deposit (Proffett, 2009), and\nprecipitation of sulfides in structurally focused zones as a con-sequence of cooling, interaction with wall rocks, boiling,\nand/or mixing with meteoric waters (Gustafson a

In [ ]:
pdf_path = os.path.join(
    folder_path, 'The_Bingham_Canyon_Porphyry_Cu_Mo_Au_Dep.pdf')
image_fps = extract_images_from_pdf(pdf_path, output_folder=os.path.join(
    folder_path, 'extracted_images'))
len(image_fps), image_fps

(25,
 ['/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_6_1.png',
  '/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_6_2.png',
  '/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_6_3.png',
  '/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_6_4.png',
  '/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_6_5.png',
  '/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_6_6.png',
  '/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_6_7.png',
  '/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_6_8.png',
  '/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_8_1.png',
  '/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_8_2.png',
  '/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_8_3.png',
  '/content/drive/MyDrive/Colab Notebooks/extracted_images/image_page_8_4.png',
  '/content/drive/MyDrive/Colab Not